In [17]:
import numpy as np
import glob
import datetime

In [18]:
dataset = []
for idx in glob.glob("data/*.csv"):
    if idx == 'data/S&P500.csv': continue
    stock = np.genfromtxt(idx,skip_header=1,usecols=(0,1),delimiter=',',dtype=["S10","f8"])
    data = np.zeros((stock.shape[0],2))
    data[:,0] = np.array([datetime.datetime.strptime(entry[0], "%Y-%m-%d").date().timetuple().tm_yday for entry in stock])
    data[:,1] = np.array([entry[1] for entry in stock])
    dataset.append((idx,data))

In [19]:
num_stocks = 4
data_batches = []
idxs = []

for stock1 in range(0,len(dataset)):
    for stock2 in range(stock1+1,len(dataset)):
        for stock3 in range(stock2+1,len(dataset)):
            for stock4 in range(stock3+1,len(dataset)):
                size = np.min([dataset[stock1][1].shape[0],dataset[stock2][1].shape[0],dataset[stock3][1].shape[0],dataset[stock4][1].shape[0]])
                data_batch = np.hstack((dataset[stock1][1][0:size,:],dataset[stock2][1][0:size,:],dataset[stock3][1][0:size,:],dataset[stock4][1][0:size,:]))
                data_batches.append(data_batch)
                if not idxs:
                    idxs.append(data_batch.shape[0]-2)
                else:
                    idxs.append(idxs[-1] + data_batch.shape[0] - 1)

In [22]:
idxs

[1128,
 4209,
 9116,
 14023,
 18930,
 20536,
 21665,
 22794,
 23923,
 25052,
 26181,
 29262,
 32343,
 35424,
 37030,
 41937,
 46844,
 48450,
 53357,
 54963,
 56569,
 57698,
 58827,
 59956,
 61085,
 62214,
 65295,
 68376,
 71457,
 73063,
 79801,
 86539,
 88145,
 94883,
 96489,
 98095,
 99224,
 100353,
 101482,
 102611,
 103740,
 104869,
 105998,
 107127,
 108256,
 109385,
 112466,
 115547,
 117153,
 120234,
 121840,
 123446,
 131179,
 132785,
 134391,
 135997,
 137126,
 138255,
 139384,
 140513,
 141642,
 144723,
 147804,
 150885,
 152491,
 157398,
 162305,
 163911,
 168818,
 170424,
 172030,
 173159,
 174288,
 175417,
 176546,
 177675,
 178804,
 179933,
 181062,
 182191,
 183320,
 186401,
 189482,
 191088,
 194169,
 195775,
 197381,
 202288,
 203894,
 205500,
 207106,
 208235,
 209364,
 210493,
 211622,
 212751,
 213880,
 215009,
 216138,
 217267,
 218396,
 221477,
 224558,
 226164,
 229245,
 230851,
 232457,
 239195,
 240801,
 242407,
 244013,
 245142,
 246271,
 247400,
 248529,
 2496

In [4]:
def label(current, future):
    label = np.zeros(5) # [stock1, stock2, stock3, stock4, Cash]
    profit = np.true_divide(future - current, current)
    idx = np.argmax(profit)
    if profit[idx] <= 0: idx = -1
    label[idx] = 1
    return label

In [5]:
label_batches = []

for batch in data_batches:
    label_batch = np.zeros((batch.shape[0]-1,5))
    for idx in range(batch.shape[0]-1):
        current, future = batch[idx+1][[1,3,5,7]], batch[idx][[1,3,5,7]]
        label_batch[idx] = label(current,future)
    label_batches.append(label_batch)

In [6]:
X_train = np.vstack((batch[1:,:] for batch in data_batches))

In [7]:
X_train.shape

(264809, 8)

In [8]:
Y_train = np.vstack(np.vstack((label for label in label_batch)) for label_batch in label_batches)

In [9]:
Y_train.shape

(264809, 5)

In [10]:
np.save('X_train',X_train)
np.save('Y_train',Y_train)

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM

Using TensorFlow backend.


In [13]:
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train_norm = (X_train - mean)/std

In [20]:
input_dim = X_train.shape[1]
output_dim = Y_train.shape[1]
T = 100
X_train_seq = []
Y_train_seq = []
for i in range(len(idxs)):
    if i == 0:
        start_idx = 0
    else:
        start_idx = idxs[i-1] + 1
    end_idx = idxs[i]
    for j in range(start_idx + T - 1, end_idx+1):
        tX = X_train_norm[j - (T - 1): j + 1]
        tY = Y_train[j]
        X_train_seq.append(tX[None,:,:])
        Y_train_seq.append(tY[None,:])
        del tX
        del tY

X_train_seq = np.concatenate(X_train_seq, axis=0)
Y_train_seq = np.concatenate(Y_train_seq, axis=0)

In [27]:
data_dim = 8
timesteps = 100
nb_classes = 5

model = Sequential()
#model.add(Dense(42, input_dim=8))
model.add(LSTM(100, return_sequences=False, input_shape=(timesteps, data_dim)))
#model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_crossentropy'])

model.fit(X_train_seq[:300], Y_train_seq[:300], batch_size=50, nb_epoch=20)
#score = model.evaluate(X_test, Y_test, batch_size=16)

Epoch 1/20
300/300 [==============================] - 2s - loss: 1.6707 - categorical_crossentropy: 1.6707     
Epoch 2/20
300/300 [==============================] - 1s - loss: 1.6119 - categorical_crossentropy: 1.6119     
Epoch 3/20
300/300 [==============================] - 2s - loss: 1.6006 - categorical_crossentropy: 1.6006     
Epoch 4/20
300/300 [==============================] - 2s - loss: 1.5888 - categorical_crossentropy: 1.5888     
Epoch 5/20
300/300 [==============================] - 2s - loss: 1.5845 - categorical_crossentropy: 1.5845     
Epoch 6/20
300/300 [==============================] - 2s - loss: 1.5825 - categorical_crossentropy: 1.5825     
Epoch 7/20
300/300 [==============================] - 2s - loss: 1.5678 - categorical_crossentropy: 1.5678     
Epoch 8/20
300/300 [==============================] - 2s - loss: 1.5702 - categorical_crossentropy: 1.5702     
Epoch 9/20
300/300 [==============================] - 2s - loss: 1.5695 - categorical_crossentropy: 1.56

In [105]:
model.fit(X_train_seq[:50], Y_train_seq[:50], batch_size=25, nb_epoch=100)

Epoch 1/100
50/50 [==============================] - 0s - loss: 0.4325 - categorical_crossentropy: 0.4325     
Epoch 2/100
50/50 [==============================] - 0s - loss: 0.4656 - categorical_crossentropy: 0.4656     
Epoch 3/100
50/50 [==============================] - 0s - loss: 0.7490 - categorical_crossentropy: 0.7490     
Epoch 4/100
50/50 [==============================] - 0s - loss: 1.4824 - categorical_crossentropy: 1.4824     
Epoch 5/100
50/50 [==============================] - 0s - loss: 0.4298 - categorical_crossentropy: 0.4298     
Epoch 6/100
50/50 [==============================] - 0s - loss: 0.3367 - categorical_crossentropy: 0.3367     
Epoch 7/100
50/50 [==============================] - 0s - loss: 0.3191 - categorical_crossentropy: 0.3191     
Epoch 8/100
50/50 [==============================] - 0s - loss: 0.3330 - categorical_crossentropy: 0.3330     
Epoch 9/100
50/50 [==============================] - 0s - loss: 0.3089 - categorical_crossentropy: 0.3089     
E

In [108]:
predictions = np.argmax(model.predict_proba(X_train_seq[:50,:,:]),axis=1)
truth = np.argmax(Y_train_seq[:50],axis=1)
np.mean(predictions==truth)

50/50 [==============================] - 0s     


0.78000000000000003